## After GPR-Lidar Fusion

In [ ]:
import datetime
import pandas as pd
import numpy as np

## Identify the multiple GPR points within LiDAR cells and find median TWTs
 - be sure to update CSV name
 - remoce random IX and lidar SD column when proper data updated
 - update column names 

In [ ]:
#import data change name based on csv file
GPR = pd.read_csv('GPR_ALL.csv')
GPR.head()

# %whos - this will tell you what variables you have 

#filter by date to seperate 2 data sets
date1 = '2020-02-02'
date2 = '2020-02-04'
GPR['date'] = pd.to_datetime(GPR['date'])
GPR1 = GPR[(GPR['date'] <= date1)] #less than and equal to
GPR1.head()

GPR2 = GPR[(GPR['date'] >= date2)]  #greater than and equal to this date 

print(len(GPR1), ',', len(GPR2))

### THIS IS TEMPORARYYY!! Delete when we have real data
##add fake columns for GPR to test out lidar 
#will delete when have real data

lidarSD = np.ones((len(GPR1),1))
GPR1['lidarSD'] = lidarSD
GPR1.head()

Ix = range(500000)
Ix = np.random.choice(Ix, size=len(GPR1))
GPR1['Ix'] = Ix

#### KEEP HERE
## update columns when we know them with real data ( Ix column is refering to the Lidar cell ID)
#GPRLiDAR = GPR1.groupby(['Ix']).median()

#del GPRLiDAR['easting']
#del GPRLiDAR['northing']
#del GPRLiDAR['Unnamed: 0']

#GPRLiDAR.drop('easting',inplace = True)
#GPRLiDAR.head()
#length = len(GPRtest)
#print(length)
#GPRtest = GPR1.groupby(['Ix']).agg({'twt': 'median'})
#GPRtest = GPR1.groupby(['date','Ix','geom','easting','northing']).count().reset_index()
#GPRtest = GPR1.groupby(['date'],as_index=False).agg({'date':'first','Ix':'median','geom':'first'}).count().reset_index()

GPR1.head()
#len(GPRLiDAR)

In [ ]:
#GPRLiDAR.drop('easting',inplace = True)

GPRtest = GPR1.groupby(['Ix', 'date','easting','northing']).agg({'twt': 'median'})
#GPRtest = GPR1.groupby(['Ix', 'date','easting','northing']).agg({'twt': 'median'})

length = len(GPRtest)
#print(length)
GPRtest.head()

In [ ]:
GPR1.head()

## Calculate snow density 
- eq 1. hs = TWT(v)/2  hs = Lidar snow depth, TWT = gpr two way travel time, V = wave speed
- eq 2. V = C/ sqrt(e),  V = wave speed, C = speed of light (0.3m/ns), e = dielectric permittivity
- eq 3. e = (1+0.845d)^2  Kovacs et al. 1995 d = density (kg/m3 /1000)

re-arrange eq 1.
- V = hs(2)/TWT

re-arrange eq 2.
- e = C^2/V^2

re-arrange eq 3.
- d = (sqrt(e)-1)/0.845

- update lidarSD column name 

In [ ]:
#First calculation 1
#V = wave speed
GPRLiDAR['v']= (GPRLiDAR["lidarSD"]*2)/GPR["twt"]

In [ ]:
#second Calculation 
#e = dielectric permitivity
GPRLiDAR['e'] = 0.3**2/(GPRLiDAR['v']**2)

In [ ]:
#third calculation
# density
# d = (sqrt(e)-1)/0.845
GPRLiDAR['dens'] = (((GPRLiDAR['e'])**(1/2))-1)/0.845
#convert units to kg/m^3
GPRLiDAR['dens'] = GPRLiDAR['dens']*1000

In [ ]:
GPRLiDAR.head()

## Simple Median Filter to remove 
- running median filter using a buffer


In [ ]:
GPRLiDAR.head()

## Calculate SWE
- GPR density (kg/m3)
- LiDAR Snow Depth (cm)

- SWE (cm) = LiDAR SD (cm) * GPR Density (g/cm3)

In [ ]:
GPRLiDAR['SWE'] = GPRLiDAR['lidarSD'] * (GPRLiDAR['dens']/1000)
GPRLiDAR.head()

## Export CSV with new values
Table should include:
- ASO X 
- ASO Y
- ASO snow depth
- ASO SWE
- GPR TWT
- GPR Density
- GPR/LiDAR SWE



In [ ]:
GPRLiDAR.to_csv('GPRLiDAR1.csv')